# Uczenie reprezentacji: projekt

## Temat: Wykrywanie społeczności w sieciach

**Autorzy:** Dominika Szydło (250109), Oliwier Kaszyca (250113)

---

In [1]:
import pandas as pd
from pathlib import Path
from gensim.models import Word2Vec
from IPython.display import Code, display

from src.utils import generate_summary

In [2]:
DATA_PATH = Path("data")
EMBBEDDINGS_PATH = Path("embbeddings")
RESULTS_PATH = Path("results")

---

# Opis problemu

## Typ problemu

Rozważany problem to klasyfikacja wierzchołków w grafie reprezentującym sieć społeczną. 


## Zbiór danych


Badania przeprowadzono na zmodyfikowanej sieci *email-Eu-core* składającej się z 986 wierzchołków, przyporządkowanych do 37 grup. Zbiór danych reprezentuje konwersacje mailowe między pracownikami pewnej firmy. Krawędź między pracownikami oznacza wysłanie maila przez pracownika reprezentowanego przez wierzchołek, od którego wychodzi krawędź, do pracownika reprezentowanego przez wierzchołek do ktorego krawęź wchodzi. Grupy wierzchołków oznaczają departamenty, do których należą pracownicy. Z sieci została usunięta kierunkowość, w celu otrzymania prostej relacji między pracownikami. Wizualizację badanej sieci przedstawiono poniżej. 

<center><img src="images/network.png" width="600" height="300"></center>

Badany zbiór danych charakteryzuje się wysokim stopniem niezbalansowania, co wpłynęło na sposób przygotowania eksperymentów.

<center><img src="images/departments_cleaned.png"A width="600" height="300"><center>

### Dane wejściowe

Danymi wejściowymi będzie informacja o wierzchołku, reprezentowana jako wektor wygenerowany przez metodę [`node2vec`](https://github.com/aditya-grover/node2vec) ($X$) oraz informacja o przynależności do działu ($Y$).

In [3]:
pd.read_csv(DATA_PATH / "train_data_64.tsv", sep=" ")

,user_id,department_id,embbedding_id
0,0,1,274
1,1,1,214
2,2,19,62
3,3,19,121
4,4,19,63
...,...,...,...
971,1000,4,746
972,1001,19,670
973,1002,1,908
974,1003,6,912


In [4]:
model = Word2Vec.load(str(EMBBEDDINGS_PATH / "graph_64.model"))
model.wv[0]

array([ 0.2546108 , -0.23057798,  0.21836537,  0.46526358, -0.17184058,
        0.08660654, -0.1691932 ,  0.22447565, -0.04186373, -0.10044232,
        0.09406602, -0.14495793, -0.18923284,  0.12602815,  0.02033304,
        0.13001777, -0.03674696, -0.15397125, -0.0108666 ,  0.1237722 ,
        0.20768197,  0.02528708,  0.06163227,  0.05012866, -0.0123972 ,
       -0.10703237, -0.22338434, -0.17893176, -0.04474849,  0.24680692,
       -0.23790954,  0.17568135, -0.18129091, -0.04467465,  0.12230913,
       -0.10467809,  0.14315628,  0.06247655,  0.01302138,  0.35156024,
       -0.3315806 , -0.28147262,  0.19358933, -0.3252379 , -0.16074616,
       -0.29917642,  0.16210511,  0.01038771, -0.09245422,  0.21811223,
       -0.08821449,  0.02596824, -0.05547078,  0.09915774, -0.07835271,
        0.06661233, -0.02587465,  0.13907675, -0.1931055 ,  0.04680524,
       -0.0336911 ,  0.15107258, -0.00409859,  0.22574055], dtype=float32)

Ważnym krokiem w tworzeniu wektorów osadzeń było przygotowanie grafu, który reprezentuje sieć w badanym zbiorze. Wierzchołki w stworzonym grafie, w wersji podstawowej zawierały informację o przynależności do danego działu. Aby uniknąć niekontrolowanego wycieku informacji, ta cecha została usunięta tuż przed wywołaniem metody generującej wektory osadzeń. Ostatecznie graf posiadał tylko dane na temat połączeń między użytkownikami, a cecha mówiąca o dziale została przypisana do niezależnego zbioru $Y$. 

### Dane wyjściowe

Jako dane wyjściowe otrzymujemy identyfikator klasy $c$, który określa przynależność pracownika do działu (klasy).

## Hipoteza badawcza

_Wykorzystanie informacji o strukturze grafu w trakcie uczenia modelu wpływa pozytywnie na otrzymywane wyniki._

# Opis wybranych modeli

## Graph Attention Network (GAT)

Model GAT został opublikowany w [pracy](https://arxiv.org/pdf/1710.10903.pdf) napisane przez Petara Velickovica w 2017 roku. Nowością, jaką wprowadził, było zastosowanie mechanizmu uwagi, znanego z przetwarzania języka naturalnego, w grafowych sieciach neuronowych. Dzięki temu mechanizmowi wierzchołki grafu są w stanie przypisywać (nie wprost) wagi cechom ich sąsiadów, co pozytywnie wpływa na osiągane wyniki. W GAT reguła propagacji ma następującą postać:

$$h^{(l+1)}_u = \alpha_{u,u} W^{(l)}h^{(l)}_u + \sum_{v \in \mathcal{N}(u)} \alpha_{u,v}W^{(l)}h^{(l)}_v $$

$$\alpha_{i, j} = \frac{\exp(\text{LeakyReLU}(a^T[W^{(l)}h^{(l)}_i || W^{(l)}h^{(l)}_j]))}{\sum_{k\in i\cup \mathcal{N}(i)} \exp(\text{LeakyReLU}(a^T[W^{(l)}h^{(l)}_i || W^{(l)}h^{(l)}_k]))},$$

gdzie:
- $W^{(l)}$ to wyuczalna macierz parametrów
- $a$ to wyuczalne parametry mechanizmu uwagi (ang. *attention parameters*)
- $\alpha$ to współczynniki mechanizmu uwagi (ang. *attention coefficients*)

Metoda pozwala również na wykorzystanie wielu głowic uwagi (ang. *multi-headed attention*).

In [ ]:
class GATModel(nn.Module):
    def __init__(
        self,
        in_dim: int,
        hidden_dim: int,
        out_dim: int
    ):
        super().__init__()
        self.conv1 = GATConv(in_dim, hidden_dim, heads=1)
        self.act1 = nn.ReLU()
        self.conv2 = GATConv(hidden_dim, out_dim, heads=1)
        self.act2 = nn.ReLU()

    def forward(self, x, edge_index):
        z = self.act1(self.conv1(x, edge_index))
        z = self.act2(self.conv2(z, edge_index))
        return z

## Graph Convolutional Network (GCN) 

Grafowe sieci neuronowe znane są od lat, jednak dopiero rozwój i popularyzacja uczenia głębokiego pozwoliła na efektywne implementacje. Najpopularniejszym obecnie modelem grafowej sieci neuronowej jest **grafowa konwolucja** (GCN - *Graph Convolutional Network*), która została zaproponowana przez Kipfa w 2016 roku – [artykuł](https://arxiv.org/pdf/1609.02907.pdf). Praca ma już ponad 11 tysięcy cytowań i wiele obecnych GNNów jest oparta na niej. Zaproponowana sieć zostanie wykorzystana w scenariuszu nadzorowanym.

GCN w każdej zdefiniowanej warstwie oblicza nowe cechy wierzchołków $H^{(l+1)}$ na podstawie cech istniejących $H^{(l)}$. Bazując na wzorze wprowadzonym na laboratorium 4, obliczanie cech wykonywane jest następująco

$$H^{(l+1)} = \hat{D}^{-\frac{1}{2}}\hat{A}\hat{D}^{-\frac{1}{2}}H^{(l)}W^{(l)},$$
gdzie:
- $\hat{A} = A + I$ to macierz sąsiedztwa grafu z dołączonymi pętlami zwrotnymi na każdym wierzchołku (krawędź z danego wierzchołka do samego siebie)
- $\hat{D}$ to macierz stopnii węzłów (macierz diagonalna)
- $\hat{D}^{-\frac{1}{2}}\hat{A}\hat{D}^{-\frac{1}{2}}$ to tzw. symetryczna normalizacja macierzy sąsiedztwa
- $W^{(l)}$ to macierz wyuczalnych parametrów

Elementem kluczowym jest dodanie pętli na każdym wierzchołku, dzięki czemu osiąga się uśrednione cechy zarówno sąsiadów jak i danego wierzchołka. Dodatkowe informacje dostarcza symetryczna normalizacja, która uwzględnia stopień danego wierzchołka oraz stopień sąsiada.

In [ ]:
class GCNModel(nn.Module):
    def __init__(
        self,
        in_dim: int,
        hidden_dim: int,
        out_dim: int
    ):
        super().__init__()
        self.conv1 = GCNConv(in_dim, hidden_dim)
        self.act1 = nn.ReLU()
        self.conv2 = GCNConv(hidden_dim, out_dim)
        self.act2 = nn.Softmax(dim=1)

    def forward(self, x, edge_index):
        z = self.act1(self.conv1(x, edge_index))
        z = self.act2(self.conv2(z, edge_index))
        return z

class GCNModel(nn.Module):
    def __init__(
        self,
        in_dim: int,
        hidden_dim: int,
        out_dim: int
    ):
        super().__init__()
        self.conv1 = GCNConv(in_dim, hidden_dim)
        self.act1 = nn.ReLU()
        self.conv2 = GCNConv(hidden_dim, out_dim)
        self.act2 = nn.Softmax(dim=1)

    def forward(self, x, edge_index):
        z = self.act1(self.conv1(x, edge_index))
        z = self.act2(self.conv2(z, edge_index))
        return z

Wykorzystany GCN posiada dwie warstwy oddzielone funkcją aktywacji ReLU i warstwą Softmax na końcu. W eksperymentach rozmiar sieci był zmieniany, co zostanie opisane w sekcji **Opis rezultatów**.

# Eksperymenty

Eksperymenty przeprowoadzono osobno dla dwóch modeli oraz dla dwóch rozmiarów wektorów osadzeń wierzchołków - 64 i 128. 

### Zbiór danych

Dane do eksperymentów zostały zapisane jako dataset w formacie `pytorch geometric` - `InMemoryDataset`. Oprócz stworzonych wektorów osadzeń, na danych nie zastosowano modyfikacji oraz transformacji. 

In [9]:
display(Code("src/dataset.py"))

import os
from typing import Callable, List, Optional

import torch
import pandas as pd

from torch_geometric.data import (
    Data,
    InMemoryDataset,
)

from networkx import Graph
from gensim.models import KeyedVectors
from torch_geometric.utils.convert import from_networkx
import networkx as nx
from sklearn.model_selection import train_test_split

class EmailEUCore(InMemoryDataset):
    r"""A modified e-mail communication network of a large European research
    institution, taken from the `"Local Higher-order Graph Clustering"
    <https://www-cs.stanford.edu/~jure/pubs/mappr-kdd17.pdf>`_ paper.
    Nodes indicate members of the institution.
    An edge between a pair of members indicates that they exchanged at least
    one email.
    Node labels indicate membership to one of the 37 departments.

    Args:
        root (string): Root directory where the dataset should be saved.
        transform (callable, optional): A function/transform that takes in an
            :obj:`torch_geometric.data.Data` object and returns a transformed
            version. The data object will be transformed before every access.
            (default: :obj:`None`)
        pre_transform (callable, optional): A function/transform that takes in
            an :obj:`torch_geometric.data.Data` object and returns a
            transformed version. The data object will be transformed before
            being saved to disk. (default: :obj:`None`)
    """

    def __init__(
        self,
        root: str,
        embbeddings_path: str,
        data_path: str,
        graph_path: str,
        split: str = "predefined",
        transform: Optional[Callable] = None,
        pre_transform: Optional[Callable] = None,
        random_state: int = 345,
        num_train_per_class: int = 5
    ):
        self._embbeddings_path = embbeddings_path
        self._graph_path = graph_path
        self._data_path = data_path
        self._random_state = random_state
        
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
        self.split = split
        
        assert self.split in ['predefined', 'full']

        if split == 'full':
            data = self.get(0)
            data.train_mask.fill_(True)
            data.train_mask[data.val_mask | data.test_mask] = False
            self.data, self.slices = self.collate([data])
   
    @property
    def processed_file_names(self) -> str:
        return 'data.pt'
    
    @property
    def processed_dir(self) -> str:
        return self.root
    
    @property
    def num_classes(self) -> int:
        r"""Returns the number of classes in the dataset."""
        y = self.data.y
        if y is None:
            return 0
        elif y.numel() == y.size(0):
            return torch.unique(y).numel()
        else:
            return self.data.y.size(-1)
    
    def process(self):
        
        labels = pd.read_csv(self._data_path, sep = " ")
        vectors = KeyedVectors.load_word2vec_format(
            self._embbeddings_path,
            binary=False
        )
        
        G = nx.read_gpickle(self._graph_path)
        attribs = list(G.nodes[0].keys())
        
        
        if "department_id" not in attribs:
            raise ValueError("department_id or user_id not found in node attributes")
        
        if "user_id" not in list(labels.columns):
            raise ValueError("user_id not found in train_data")
            
        train_ids, test_ids = train_test_split(
            list(labels["user_id"]),
            test_size=0.5,
            stratify=list(labels["department_id"]), 
            random_state=self._random_state
        )
        
        test_ids, valid_ids = train_test_split(
            test_ids,
            test_size=0.5,
            stratify=[labels[labels["user_id"] == y]["department_id"].iloc[0] for y in test_ids], 
            random_state=self._random_state
        )
        
        
        nx.set

W metodzie `process` dokonano połączenia wszystkich wygenerowanych informacji otrzymując obiekt `Data` o następujących polach:
- `x` - wektor osadzeń przypisany do wierzchołka
- `y` - etykieta zespołu
- `train_mask` - maska wybierająca wierzchołki do zbioru treningowego
- `test_mask` - maska wybierająca wierzchołki do zbioru testowego
- `val_mask` - maska wybierająca wierzchołki do zbioru walidacyjnego

Podział zbioru został dokonany w stosunku 50:25:25, ze względu na obecność klas o bardzo małym procencie pokrycia.

### Metryki i ewaluacja

Eksperymenty będą analizowane pod kątem następujących metryk:
- uśredniona miara AUC,
- uśredniona miara F1,
- silhouette score,
- Davies-Bouldin score.

Dodatkowo zostanie wykonana wizualizacja wektorów osadzeń z użyciem PCA i UMAPa.

Ostateczne wyniki będą porównywane na zbiorze testowym.

### Badanie hiperparametrów

Zarówno dla modelu GAT jak i GCN zostanie przeprowadzone badanie wpływu rozmiaru ich warstw ukrytych. Badanie to zostało przeprowadzone dla rozmiaru osadzen z `node2vec` o rozmiarze $64$. Eksperyment ten można znaleźć w zeszytach `GAT.ipynb` oraz `GCN.ipynb`. Oprócz tego dla modelu baseline'owego oraz GCN zostaną przeprowadzone badania wpływu rozmiaru osadzeń generowanych przez metodę `node2vec`.

### Baseline

Do porównania testowanych modeli wykorzystano prostą sieć składającą się z dwóch warstw liniowych. Szczegóły implementacyjne znajdują się w zeszycie `Baseline.ipynb`. Sieć ta była uczona na wcześniej wygenerowanych wektorach osadzeń i tylko z taką informacją miala zadanie predykować klasy wierzchołków.

In [ ]:
class Baseline(nn.Module):
    def __init__(
        self,
        in_dim: int,
        hidden_dim: int,
        num_classes: int,
        name: str = "baseline"
    ):
        super().__init__()
        self.name = name
        self.linear1 = nn.Linear(in_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_dim, num_classes)
        self.soft = nn.Softmax(dim=1)

    def predict(self, x):
        with torch.no_grad():
            return self(x).argmax(dim=1)

    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.soft(self.linear2(x))
        return x

# Opis rezultatów

W przeprowadzonych eksperymentach zbadano różne struktury sieci GAT i GCN. Następnie przebadano zachowanie wybranej struktury na różny rozmiar wektorów osadzeń. Badanie to powtórzono na baseline, co miało stanowić punkt odniesienia do wyników otrzymywanych przez GCN. Wyniki przedstawiono poniżej.

## Baseline

In [5]:
df = pd.read_csv(RESULTS_PATH / "baseline_stats.csv")
df.sort_values(by="test_f1", ascending=False).head(5)

,test_auc,test_f1,model_name
46,0.907433,0.682160,Baseline_32
66,0.900202,0.670135,Baseline_128
63,0.909134,0.660158,Baseline_128
43,0.910359,0.658958,Baseline_32
62,0.886434,0.657609,Baseline_128


In [9]:
generate_summary(df)

,f1_test,auc_test,silhoute,davies-bouldin
Supervised_GCN_64,0.649 +- 0.023,0.919 +- 0.012,0.487 +- 0.029,0.977 +- 0.044
Supervised_GCN_128,0.674 +- 0.018,0.926 +- 0.013,0.453 +- 0.027,0.973 +- 0.036


## GCN

Wyniki modelu na zbiorze:

In [10]:
df = pd.read_csv(RESULTS_PATH / "gcn_stats.csv")
df.sort_values(by="f1_test", ascending=False).head(5)

,model_name,f1_test,auc_test,silhoute,davies-bouldin
10,Supervised_GCN_128,0.714371,0.934002,0.466962,0.960949
18,Supervised_GCN_128,0.688403,0.943589,0.480429,1.007799
17,Supervised_GCN_128,0.684375,0.931982,0.441306,0.927863
1,Supervised_GCN_64,0.680867,0.942957,0.510175,0.934839
5,Supervised_GCN_64,0.676853,0.923104,0.500258,1.077800


In [11]:
generate_summary(df)

,f1_test,auc_test,silhoute,davies-bouldin
Supervised_GCN_64,0.649 +- 0.023,0.919 +- 0.012,0.487 +- 0.029,0.977 +- 0.044
Supervised_GCN_128,0.674 +- 0.018,0.926 +- 0.013,0.453 +- 0.027,0.973 +- 0.036


Wizualizacja przestrzeni osadzeń dla modelu uczonego na osadzeniach `node2vec` o rozmiarze $128$:

<center><img src="images/gcn_emb_space_128.png" width="1200" height="300"></center>

**Badanie rozmiaru warstwy ukrytej modelu przeprowadzono na wektorach osadzeń o rozmiarze 64.**

In [12]:
df = pd.read_csv(RESULTS_PATH / "gcn_experiment_hidden_dim.csv")
df.sort_values(by="f1_test", ascending=False).head(5)

,model_name,f1_test,auc_test,silhoute,davies-bouldin
20,GCN_64_hd_256,0.652489,0.902628,0.480045,0.893773
19,GCN_64_hd_128,0.638256,0.918727,0.404092,0.941212
18,GCN_64_hd_128,0.634416,0.939023,0.448258,0.911473
31,GCN_64_hd_512,0.617821,0.923005,0.459401,0.968790
9,GCN_64_hd_64,0.606218,0.901353,0.467757,0.957700


In [13]:
generate_summary(df)

,f1_test,auc_test,silhoute,davies-bouldin
GCN_64_hd_64,0.555 +- 0.04,0.915 +- 0.014,0.474 +- 0.031,0.951 +- 0.043
GCN_64_hd_128,0.55 +- 0.065,0.918 +- 0.011,0.495 +- 0.049,0.915 +- 0.025
GCN_64_hd_256,0.545 +- 0.056,0.909 +- 0.01,0.491 +- 0.044,0.925 +- 0.04
GCN_64_hd_512,0.563 +- 0.036,0.915 +- 0.011,0.487 +- 0.03,0.893 +- 0.039


Jak można zauważyć, rozmiar warstwy ukrytej nie ma znaczącego wpływu na otrzymywane wyniki.

**Badanie wpływu rozmiaru wektora osadzeń na ostateczne wyniki**

In [14]:
df = pd.read_csv(RESULTS_PATH / "gcn_stats_embbeddings.csv")
df.sort_values(by="f1_test", ascending=False).head(5)

,model_name,f1_test,auc_test,silhoute,davies-bouldin
46,Supervised_GCN_32,0.638395,0.920099,0.398425,0.996269
33,Supervised_GCN_16,0.606022,0.939272,0.449879,0.934757
49,Supervised_GCN_32,0.605823,0.923571,0.481846,0.954384
40,Supervised_GCN_32,0.596814,0.933209,0.463917,1.135395
47,Supervised_GCN_32,0.589125,0.930091,0.387472,0.949482


Dokładne omówienie wyników otrzymywanych w tym eksperymencie zostanie zaprezentowane w sekcji porównania z baselinem.

## GAT

Wyniki modelu na zbiorze:

In [15]:
df = pd.read_csv(RESULTS_PATH / "gat_stats.csv")
df.sort_values(by="f1_test", ascending=False).head(5)

,model_name,f1_test,auc_test,silhoute,davies-bouldin
3,GAT_emb_dim_64,0.739385,0.937540,0.293423,1.054201
6,GAT_emb_dim_128,0.738659,0.937896,0.308066,1.142875
4,GAT_emb_dim_64,0.737558,0.930167,0.283656,1.099215
8,GAT_emb_dim_128,0.730467,0.898785,0.306579,1.011764
0,GAT_emb_dim_64,0.715054,0.912775,0.308584,1.090976


In [16]:
generate_summary(df)

,f1_test,auc_test,silhoute,davies-bouldin
GAT_emb_dim_64,0.718 +- 0.021,0.922 +- 0.012,0.294 +- 0.018,1.099 +- 0.053
GAT_emb_dim_128,0.707 +- 0.029,0.906 +- 0.021,0.289 +- 0.031,1.109 +- 0.116


Wizualizacja przestrzeni osadzeń dla modelu uczonego na osadzeniach `node2vec` o rozmiarze $128$:

<center><img src="images/gat_emb_space_128.png" width="1200" height="600"></center>

**Badanie rozmiaru warstwy ukrytej modelu:**

In [17]:
df = pd.read_csv(RESULTS_PATH / "gat_experiment_hidden_dim.csv")
df.sort_values(by="f1_test", ascending=False).head(5)

,model_name,f1_test,auc_test,silhoute,davies-bouldin
15,GAT_64_hd_512,0.748927,0.913028,0.336721,1.130423
16,GAT_64_hd_512,0.732445,0.914467,0.305496,1.073214
1,GAT_64_hd_64,0.731746,0.919056,0.307029,1.130864
5,GAT_64_hd_128,0.722876,0.927141,0.293133,1.159511
11,GAT_64_hd_256,0.722156,0.917546,0.322765,1.142573


In [18]:
generate_summary(df)

,f1_test,auc_test,silhoute,davies-bouldin
GAT_64_hd_64,0.707 +- 0.018,0.915 +- 0.007,0.313 +- 0.011,1.127 +- 0.023
GAT_64_hd_128,0.705 +- 0.017,0.925 +- 0.009,0.31 +- 0.011,1.132 +- 0.044
GAT_64_hd_256,0.702 +- 0.018,0.921 +- 0.01,0.322 +- 0.004,1.105 +- 0.035
GAT_64_hd_512,0.717 +- 0.022,0.92 +- 0.007,0.308 +- 0.018,1.13 +- 0.052


## Porównanie

### Baseline vs GCN

In [19]:
df = pd.concat([
    pd.read_csv(RESULTS_PATH / "gcn_stats.csv"),
    pd.read_csv(RESULTS_PATH / "gcn_stats_embbeddings.csv"),
    pd.read_csv(RESULTS_PATH / "baseline_stats.csv").rename(
        columns={"test_f1": "f1_test", "test_auc": "auc_test"}
    )
    
])

In [20]:
generate_summary(df).sort_values(by="f1_test", ascending=False)

,f1_test,auc_test,silhoute,davies-bouldin
Supervised_GCN_128,0.674 +- 0.018,0.926 +- 0.013,0.453 +- 0.027,0.973 +- 0.036
Supervised_GCN_64,0.649 +- 0.023,0.919 +- 0.012,0.487 +- 0.029,0.977 +- 0.044
Baseline_128,0.635 +- 0.022,0.896 +- 0.007,nan +- nan,nan +- nan
Baseline_32,0.634 +- 0.026,0.903 +- 0.01,nan +- nan,nan +- nan
Baseline_16,0.62 +- 0.018,0.908 +- 0.007,nan +- nan,nan +- nan
Baseline_64,0.608 +- 0.015,0.897 +- 0.006,nan +- nan,nan +- nan
Supervised_GCN_32,0.578 +- 0.035,0.92 +- 0.009,0.424 +- 0.04,0.979 +- 0.063
Baseline_8,0.57 +- 0.03,0.881 +- 0.01,nan +- nan,nan +- nan
Supervised_GCN_16,0.555 +- 0.04,0.916 +- 0.013,0.42 +- 0.033,0.924 +- 0.037
Supervised_GCN_8,0.406 +- 0.122,0.902 +- 0.019,0.422 +- 0.034,0.884 +- 0.092


Powyższe badanie dostarcza wiele ciekawych obserwacji. Po pierwsze średni wynik wzrasta wraz ze wzrostem rozmiaru wektora osadzeń. Jest to dosyć intuicyjny wynik - im dłuższy wektor, tym więcej cech jest dostarczanych podczas uczenia. Wyjątek stanowią wektory o długości 64 dla baseline'a, dla których sieć ma gorsze wyniki niż w przypadku wektorów o długości 16 i 32.

Drugą, ciekawszą obserwacją, jest fakt, że sieć baseline do pewnego momentu osiąga lepsze f1 niż GCN. Dopiero wektory o długości większej niż 64 pozwalają uzyskać lepsze wyniki niż baseline w tej samej konfiguracji. Przykładowo dla wektorów o długości 8 baseline jest o 17 pkt procentowych lepszy!

In [21]:
generate_summary(df).sort_values(by="auc_test", ascending=False)

,f1_test,auc_test,silhoute,davies-bouldin
Supervised_GCN_128,0.674 +- 0.018,0.926 +- 0.013,0.453 +- 0.027,0.973 +- 0.036
Supervised_GCN_32,0.578 +- 0.035,0.92 +- 0.009,0.424 +- 0.04,0.979 +- 0.063
Supervised_GCN_64,0.649 +- 0.023,0.919 +- 0.012,0.487 +- 0.029,0.977 +- 0.044
Supervised_GCN_16,0.555 +- 0.04,0.916 +- 0.013,0.42 +- 0.033,0.924 +- 0.037
Baseline_16,0.62 +- 0.018,0.908 +- 0.007,nan +- nan,nan +- nan
Baseline_32,0.634 +- 0.026,0.903 +- 0.01,nan +- nan,nan +- nan
Supervised_GCN_8,0.406 +- 0.122,0.902 +- 0.019,0.422 +- 0.034,0.884 +- 0.092
Baseline_64,0.608 +- 0.015,0.897 +- 0.006,nan +- nan,nan +- nan
Baseline_128,0.635 +- 0.022,0.896 +- 0.007,nan +- nan,nan +- nan
Baseline_8,0.57 +- 0.03,0.881 +- 0.01,nan +- nan,nan +- nan


Z kolei biorąc pod uwagę auc, GCN dla każdego zestawienia okazał się lepszy. Wynik ten jednak nie dziwi - GCN jest ściśle związana z reprezentacją wektorową.

### Podsumowanie wszystkich metod

In [22]:
generate_summary(pd.concat([
    pd.read_csv(RESULTS_PATH / "baseline_stats.csv").rename(columns={"test_f1": "f1_test", "test_auc": "auc_test"}),
    pd.read_csv(RESULTS_PATH / "gcn_stats.csv"),
    pd.read_csv(RESULTS_PATH / "gcn_experiment_hidden_dim.csv"),
    pd.read_csv(RESULTS_PATH / "gcn_stats_embbeddings.csv"),
    pd.read_csv(RESULTS_PATH / "gat_stats.csv"),
    pd.read_csv(RESULTS_PATH / "gat_experiment_hidden_dim.csv"),
    pd.read_csv(RESULTS_PATH / "gat_stats.csv")
])).sort_values(by="f1_test", ascending=False).head(9)

,auc_test,f1_test,silhoute,davies-bouldin
GAT_emb_dim_64,0.922 +- 0.011,0.718 +- 0.019,0.294 +- 0.017,1.099 +- 0.05
GAT_64_hd_512,0.92 +- 0.007,0.717 +- 0.022,0.308 +- 0.018,1.13 +- 0.052
GAT_emb_dim_128,0.906 +- 0.019,0.707 +- 0.027,0.289 +- 0.029,1.109 +- 0.11
GAT_64_hd_64,0.915 +- 0.007,0.707 +- 0.018,0.313 +- 0.011,1.127 +- 0.023
GAT_64_hd_128,0.925 +- 0.009,0.705 +- 0.017,0.31 +- 0.011,1.132 +- 0.044
GAT_64_hd_256,0.921 +- 0.01,0.702 +- 0.018,0.322 +- 0.004,1.105 +- 0.035
Supervised_GCN_128,0.926 +- 0.013,0.674 +- 0.018,0.453 +- 0.027,0.973 +- 0.036
Supervised_GCN_64,0.919 +- 0.012,0.649 +- 0.023,0.487 +- 0.029,0.977 +- 0.044
Baseline_128,0.896 +- 0.007,0.635 +- 0.022,nan +- nan,nan +- nan


Zestawiając wszystkie przeprowadzone badania, na prowadzenie wychodzi GAT, który pod kątem f1 wyprzedza GCNa oraz Baseline.

## Wnioski

Postawiona hipoteza została potwierdzona wynikami. Sieci biorące pod uwagę strukturę grafową zbioru danych uzyskały lepsze wyniki w stosunku do naiwnego klasyfikatora, lepsze nawet o $8$ pkt. procentowych dla uśrednionego F1. Różnice jednak nie są aż tak duże. Prawdopodobną przyczyną jest niski poziom skomplikownia zbioru danych oraz wysoki poziom jakości reprezentacji pozyskiwanych metodą `node2vec`. Tezę tę potwierdzają badania, które pokazują, iż jakość klasyfikacji polepsza się wraz z rozmiarem wygenerowanych tą metodą osadzeń, zarówno dla baseline'u jak i sieci GCN.